In [1]:
import os
import argparse
import random
from tqdm import tqdm
import csv
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

from metrics import word_error_rate, sentence_acc
from checkpoint import load_checkpoint
from dataset import LoadEvalDataset, collate_eval_batch, START, PAD
from flags import Flags
from utils import id_to_string, get_network, get_optimizer
import easydict

ImportError: numpy.core.multiarray failed to import

In [4]:
!pip install numpy -y

Found existing installation: numpy 1.20.3
Uninstalling numpy-1.20.3:
  Successfully uninstalled numpy-1.20.3


In [ ]:
is_cuda = torch.cuda.is_available()
parser = easydict.EasyDict({
 
        "checkpoint": ["/opt/ml/p4-fr-sorry-math-but-love-you/log/attention_50/0048.pth",
                      "/opt/ml/p4-fr-sorry-math-but-love-you/log/attention_50/0043.pth"],
    
})
max_sequence = 230
batch_size = 8

In [2]:
torch.manual_seed(21) #
random.seed(21) #
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

NameError: name 'torch' is not defined

In [3]:
hardware = "cuda" if is_cuda else "cpu"
device = torch.device(hardware)

NameError: name 'is_cuda' is not defined

In [5]:
dummy_gt = "\sin " * max_sequence

In [6]:
# transformed = A.Compose(
#         [
#             A.Resize(128, 128), #
#             ToTensorV2(),
#         ]
#     )
transformed = transforms.Compose(
        [
            # Resize so all images have the same size
            transforms.Resize((128, 128)),
            transforms.ToTensor(),
        ]
    )

In [7]:
is_cuda = torch.cuda.is_available()
models = []
token_to_id_ = load_checkpoint(parser.checkpoint[0], cuda=is_cuda)["token_to_id"]
id_to_token_ = load_checkpoint(parser.checkpoint[0], cuda=is_cuda)["id_to_token"]



In [8]:
eval_dir = os.environ.get('SM_CHANNEL_EVAL', '/opt/ml/input/data/')
file_path = os.path.join(eval_dir, 'eval_dataset/input.txt')
root = os.path.join(os.path.dirname(file_path), 'images')
output_dir = os.environ.get('SM_OUTPUT_DATA_DIR', 'submit')
with open(file_path, "r") as fd:
    reader = csv.reader(fd, delimiter="\t")
    data = list(reader)
test_data = [[os.path.join(root, x[0]), x[0], dummy_gt] for x in data]
test_dataset = LoadEvalDataset(
    test_data, token_to_id_, id_to_token_, crop=False, transform=transformed,
    rgb=1 #
)
test_data_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4, #
    collate_fn=collate_eval_batch,
)

In [9]:
for parser_checkpoint in parser.checkpoint:
    checkpoint = load_checkpoint(parser_checkpoint, cuda=is_cuda)
    options = Flags(checkpoint["configs"]).get()
    model_checkpoint = checkpoint["model"]
    model = get_network(options.network, options, model_checkpoint, device, test_dataset)
    model.eval()
    models.append(model)

In [11]:
results = []
for d in tqdm(test_data_loader):
    input = d["image"].to(device)
    expected = d["truth"]["encoded"].to(device)
    decoded_values = None
    for model in models:
        output = model(input, expected, False, 0.0)
        if decoded_values is None:
            decoded_values = output.transpose(1, 2)
        else:
            decoded_values += output.transpose(1, 2)
    decoded_values /= len(models)
    _, sequence = torch.topk(decoded_values, 1, dim=1)
    sequence = sequence.squeeze(1)
    sequence_str = id_to_string(sequence, test_data_loader, do_eval=1)
#     print(sequence_str)
    for path, predicted in zip(d["file_path"], sequence_str):
        results.append((path, predicted))
        
os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, "output.csv"), "w") as w:
    for path, predicted in results:
        w.write(path + "\t" + predicted + "\n")


100%|██████████| 4/4 [00:10<00:00,  2.68s/it]


In [6]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          99G   30G   64G  32% /
tmpfs            64M     0   64M   0% /dev
tmpfs            45G     0   45G   0% /sys/fs/cgroup
shm             1.0G  156K  1.0G   1% /dev/shm
/dev/xvdb1       99G   30G   64G  32% /etc/hosts
tmpfs            45G   12K   45G   1% /proc/driver/nvidia
/dev/xvda1       48G   32G   14G  71% /usr/bin/nvidia-smi
tmpfs           8.9G   17M  8.9G   1% /run/nvidia-persistenced/socket
udev             45G     0   45G   0% /dev/nvidia0
tmpfs            45G     0   45G   0% /proc/acpi
tmpfs            45G     0   45G   0% /proc/scsi
tmpfs            45G     0   45G   0% /sys/firmware


In [7]:
!pip install timm

     |████████████████████████████████| 346 kB 2.1 MB/s eta 0:00:01


In [8]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          99G   39G   55G  42% /
tmpfs            64M     0   64M   0% /dev
tmpfs            45G     0   45G   0% /sys/fs/cgroup
shm             1.0G  103M  922M  10% /dev/shm
/dev/xvdb1       99G   39G   55G  42% /etc/hosts
tmpfs            45G   12K   45G   1% /proc/driver/nvidia
/dev/xvda1       48G   32G   14G  71% /usr/bin/nvidia-smi
tmpfs           8.9G   17M  8.9G   1% /run/nvidia-persistenced/socket
udev             45G     0   45G   0% /dev/nvidia0
tmpfs            45G     0   45G   0% /proc/acpi
tmpfs            45G     0   45G   0% /proc/scsi
tmpfs            45G     0   45G   0% /sys/firmware
